In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive

 녹화_2020_11_03_09_38_48_214.avi   images	  result2.gsheet
 녹화_2020_11_03_09_59_08_943.avi   images.txt	  result.csv
 녹화_2020_11_03_11_01_44_508.avi  'My Drive'	  testdev2017.txt
'Colab Notebooks'		    Notebooks	  videos
 Datas				    result2.csv   yolov4


In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14370, done.
remote: Total 14370 (delta 0), reused 0 (delta 0), pack-reused 14370
Receiving objects: 100% (14370/14370), 13.08 MiB | 22.77 MiB/s, done.
Resolving deltas: 100% (9768/9768), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pwd

/content/darknet


In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

In [ ]:
!mkdir data
!ls

mkdir: cannot create directory ‘data’: File exists
3rdparty	darknet			include		       results
backup		DarknetConfig.cmake.in	json_mjpeg_streams.sh  scripts
build		darknet_images.py	LICENSE		       src
build.ps1	darknet.py		Makefile	       video_yolov3.sh
build.sh	darknet_video.py	net_cam_v3.sh	       video_yolov4.sh
cfg		data			net_cam_v4.sh
cmake		image_yolov3.sh		obj
CMakeLists.txt	image_yolov4.sh		README.md


In [ ]:
%cd /content
!pwd

/content
/content


In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# OIDv4_ToolKit 이용해 예시 데이터셋 가져오기
데이터셋이 없으면 다운로드(uncomment), classes.txt 미리 준비 필요

In [ ]:
!git clone https://github.com/theAIGuysCode/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 444, done.
remote: Total 444 (delta 0), reused 0 (delta 0), pack-reused 444
Receiving objects: 100% (444/444), 34.09 MiB | 41.21 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [ ]:
!ls

darknet  drive	OIDv4_ToolKit  sample_data


In [ ]:
!pip install -r ./OIDv4_ToolKit/requirements.txt

     |████████████████████████████████| 3.5MB 21.5MB/s 
     |████████████████████████████████| 6.8MB 48.3MB/s 
     |████████████████████████████████| 552kB 50.6MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: botocore 1.19.18 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Found existing installation: rsa 4.6
    Uninstalling rsa-4.6:
      Successfully uninstalled rsa-4.6


In [ ]:
# train data download
# !cd ./OIDv4_ToolKit && python main.py downloader --classes Bicycle Bus Car 'Fire hydrant' Motorcycle  'Palm tree' 'Traffic light' --type_csv train --limit 1500

In [ ]:
# validation data download
# !cd ./OIDv4_ToolKit && python main.py downloader --classes Bicycle Bus Car 'Fire hydrant' Motorcycle  'Palm tree' 'Traffic light' --type_csv validation --limit 300

In [ ]:
# !cp /mydrive/yolov4/classes.txt ./OIDv4_ToolKit/classes.txt
# !echo -e "$(cat ./OIDv4_ToolKit/classes.txt)"

In [ ]:
# !cp -r /mydrive/yolov4/OID/OID/* /content/OIDv4_ToolKit/OID

In [ ]:
# !cd /content/OIDv4_ToolKit && python convert_annotations.py

In [ ]:
# single class
# 원래 label 형태
# Vehicle registration plate 195.999744 411.75818200000003 318.39948799999996 512.4991299999999
# Vehicle registration plate 849.5994880000001 212.67556199999999 868.7994880000001 235.86151600000002
# yolo label 형태
# 0 0.25097631249999997 0.6770647829912024 0.11953099999999997 0.14771399999999985
# 0 0.8384760625000001 0.3279700322580645 0.018750000000000044 0.033997000000000055

In [ ]:
# copy dataset to google drive
# !ls /mydrive/yolov4
# !cp -r ./OIDv4_ToolKit/OID /mydrive/yolov4
# !ls /mydrive/yolov4

In [ ]:
#assert

# copy dataset from google drive
train, validation dataset 복사

In [ ]:
%cd /content/darknet

/content/darknet


In [ ]:
!cp -r /mydrive/yolov4/OID/Dataset/train ./data/obj

In [ ]:
!cp -r /mydrive/yolov4/OID/Dataset/validation ./data/test

In [ ]:
!ls ./data

# Configuring Files for Training(.cfg, obj.names, obj.data, train.txt, test.txt)
yolov4-obj.cfg, obj.names, obj.data, generate_train.py, generate_test.py 파일을 google drive에 미리 준비

In [ ]:
!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

In [ ]:
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

In [ ]:
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
!ls data/

# Download pre-trained weights for the convolutional layers

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Train custom object detector

In [ ]:
# to not kicked from colab service
# ctrl + shift + i
# function ClickConnect(){
# console.log("Working"); 
# document
#   .querySelector('#top-toolbar > colab-connect-button')
#   .shadowRoot.querySelector('#connect')
#   .click() 
# }
# setInterval(ClickConnect,60000)

In [ ]:
#assert False

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')
!cp chart.png /mydrive/yolov4/chart.png

In [ ]:
# 오류가 나서 갑자기 종료 되었을 때
# kick off training from where it last saved
# !./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show

In [ ]:
# imShow('chart.png')
# !cp chart.png /mydrive/yolov4/chart.png

# Checking the Mean Average Precision (mAP) of Your Model

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_1000.weights

# Run Your Custom Object Detector
prediction할 이미지 predictions.jpg로 저장

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/car1.jpg -thresh 0.3
imShow('predictions.jpg')
!cp predictions.jpg /mydrive/yolov4/car1prediction.jpg